In [1]:
from pynasapower.get_data import query_power
from pynasapower.geometry import point, bbox
import datetime


In [2]:
import geopandas as gpd
from shapely.geometry import Polygon
from typing import Union

def point(x:Union[float, int], y:Union[float, int], crs:str, z:Union[float, int]=None):
    """Generate a Point GeoDataFrame to be inserted in the API client.

    Args:
        x (float): X coordinate. In case of geographical coordinates add longitude as x
        y (float): Y coordinate. In case of geographical coordinates add latitude as y
        crs (str): Reference code of coordinate system. For reference codes of the most commonly used projections, see spatialreference.org
    
    Returns:
        gpd.GeoDataFrame: Geometry as geodataframe
    """
    if (not isinstance(x, float)) and (not isinstance(x, int)):
        raise TypeError("X coordinate must be a float or int!")
    
    if (not isinstance(y, float)) and (not isinstance(y, int)):
        raise TypeError("Y coordinate must be a float")

    if z is not None:
        if (not isinstance(z, float)) and (not isinstance(z, int)):
            raise TypeError("Z coordinate must be a float or int!")
    
    if not isinstance(crs, str):
        raise TypeError("crs must be a str!")
    
    if not crs.startswith("EPSG:"):
        raise ValueError("crs must start with EPSG:... Try to find the reference code at spatialreference.org!")
    
    geometry = gpd.GeoDataFrame([], geometry=gpd.points_from_xy([x], [y], [z]), crs=crs)

    if geometry.crs != "EPSG:4326": # Force EPSG:4326 to properly interact with the API 
        geometry = geometry.to_crs("EPSG:4326")

    return geometry

def bbox(x_min:Union[float, int], x_max:Union[float, int], y_min:Union[float, int], y_max:Union[float, int], crs:str):
    """Create a bounding box polygon using minimum and maximum coordinates along with a coordinate reference system (CRS).
    This function takes minimum and maximum x and y coordinates, and a CRS code, and creates a GeoDataFrame containing a bounding box polygon.
    The function performs input validation to ensure the correctness of the input parameters.

    Parameters:
        x_min (float or int): Minimum x-coordinate of the bounding box. In case of geographical coordinates add longitude as x
        x_max (float or int): Maximum x-coordinate of the bounding box. In case of geographical coordinates add longitude as x
        y_min (float or int): Minimum y-coordinate of the bounding box. In case of geographical coordinates add latitude as y
        y_max (float or int): Maximum y-coordinate of the bounding box. In case of geographical coordinates add latitude as y
        crs (str): Coordinate Reference System (CRS) code in the format "EPSG:XXXX", where XXXX is the EPSG code representing the CRS. For reference codes of the most commonly used projections, see spatialreference.org

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing a bounding box polygon represented as a Shapely Polygon object
    """
    if (not isinstance(x_min, float)) and (not isinstance(x_min, int)):
        raise TypeError("x_min coordinate must be a float or int!")

    if (not isinstance(x_max, float)) and (not isinstance(x_max, int)):
        raise TypeError("x_max coordinate must be a float or int!")

    if (not isinstance(y_min, float)) and (not isinstance(y_min, int)):
        raise TypeError("y_min coordinate must be a float or int!")  
    
    if (not isinstance(y_max, float)) and (not isinstance(y_max, int)):
        raise TypeError("y_max coordinate must be a float or int!")

    if x_min > x_max:
        raise ValueError("max_x value cannot be smaller than x_min!")
    
    if y_min > y_max:
        raise ValueError("max_y value cannot be smaller than y_min!")
    
    if not isinstance(crs, str):
        raise TypeError("crs must be a str!")
    
    if not crs.startswith("EPSG:"):
        raise ValueError("crs must start with EPSG:... Try to find the reference code at spatialreference.org!")
    
    geometry = Polygon([(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)])

    bbox = gpd.GeoDataFrame(geometry=[geometry], crs = crs)
    
    if bbox.crs != "EPSG:4326": # Force EPSG:4326 to properly interact with the API 
        bbox = bbox.to_crs("EPSG:4326")
    
    return bbox

In [3]:
import requests
from requests import exceptions
import geopandas as gpd
import pandas as pd
from io import StringIO
import xarray as xr
import datetime
import os

DEFAULT_POWER_VARIABLES = ["ALLSKY_SFC_SW_DWN", "T2M", "T2M_MIN", "T2M_MAX", "RH2M", "WS2M", "PRECTOTCORR"]
HTTP_OK = 200 # ["TOA_SW_DWN", "ALLSKY_SFC_SW_DWN", "T2M", "T2M_MIN", "T2M_MAX", "T2MDEW", "WS2M", "PRECTOTCORR"]

def query_power(geometry:gpd.GeoDataFrame, start:datetime.date, end:datetime.date, to_file:bool = True, path:str = "./",
    community:str = "ag", parameters:list = [],  temporal_api:str = "hourly",
    spatial_api:str = "point", format:str = "csv"):
    """
    Query NASA Power API for climate data based on the specified geometry, temporal range, and parameters.

    Parameters:
        geometry (gpd.GeoDataFrame): GeoDataFrame containing either a Point or a Polygon geometry.
        start (datetime.date): Start date for the data retrieval in datetime.date format. Note in case of monthly or climatology temporal_api only the year is used.
        end (datetime.date): End date for the data retrieval in datetime.date format.Note in case of monthly or climatology temporal_api only the year is used.
        path (str, optional): Path to the directory where the retrieved data files will be saved. Default is './'.
        to_file (bool, optional): Whether to save the data to files or not. Default is True.
        community (str, optional): NASA Power community to query data from. Default is "ag".
        parameters (list, optional): List of parameter names to retrieve. Default is an empty list. If it is not provided variables TOA_SW_DWN, "ALLSKY_SFC_SW_DWN, T2M, T2M_MIN, T2M_MAX, T2MDEW, WS2M, PRECTOTCORR are downloaded.
        temporal_api (str, optional): Temporal resolution for the data (e.g., "hourly", "daily", "monthly", "climatology"). Default is "hourly".
        spatial_api (str, optional): Spatial resolution for the data (e.g., "point", "regional"). Default is "point".
        format (str, optional): Format for the retrieved data files (e.g., "csv", "netcdf", "json", "ascii"). Default is "csv".

    Returns:
        xr.Dataset or pd.DataFrame or dict: Retrieved data in dictionary format in case of format = 'json', pd.DataFrame in case of format = 'csv' or 'ascii' and xr.Dataset in case of format = 'netcdf'.

    Note:
        - The geometry parameter must be a GeoDataFrame containing either a Point or a Polygon geometry.
        - The start and end parameters must be in datetime.date format.
        - The temporal_api parameter must be one of: "hourly", "daily", "monthly", "climatology".
        - The spatial_api parameter must be one of: "point", "regional".
        - The format parameter must be one of: "csv", "netcdf", "json", "ascii".
    """

    available_temporal = ["hourly", "daily", "monthly", "climatology"]
    if temporal_api not in available_temporal:
        raise ValueError(f"Argument 'temporal_resolution' must be one of: {', '.join(t for t in available_temporal)}")
    
    available_communities = ["ag", "sb", "re"]
    if community not in available_communities:
        raise ValueError(f"Argument 'community' must be one of: {', '.join(c for c in available_communities)}")    
    
    available_spatial = ["point", "regional"] # No global because NASA does not support it
    if spatial_api not in available_spatial:
        raise ValueError(f"Argument 'spatial_api' must be one of: {', '.join(s for s in available_spatial)}")

    available_formats = ["netcdf", "ascii", "json", "csv"]
    if format not in available_formats:
        raise ValueError(f"Argument 'format' must be one of: {', '.join(f for f in available_formats)}")
    
    if temporal_api == "hourly" and spatial_api != "point":
        raise ValueError(f"For temporal_resolution = 'hourly' spatial_api can only be point!")
    
    if parameters == []:
        parameters = DEFAULT_POWER_VARIABLES

    if temporal_api == "hourly" and len(parameters)>15:
        raise ValueError("A maximum of 15 parameters can currently be requested in one submission.")
    
    if temporal_api == "daily" and len(parameters)>20:
        raise ValueError("A maximum of 20 parameters can currently be requested in one submission.")
    
    if temporal_api == "monthly" and len(parameters)>20:
        raise ValueError("A maximum of 20 parameters can currently be requested in one submission.")

    if temporal_api == "climatology" and len(parameters)>20:
        raise ValueError("A maximum of 20 parameters can currently be requested in one submission.") 

    # Tests until here

    if not isinstance(geometry, gpd.GeoDataFrame):
        raise TypeError("Argument 'geometry' must be a GeoDataframe.")
    
    if len(geometry) > 1:
        raise ValueError("Only one geometry must be provided.")
    
    if (geometry.geom_type.iloc[0] != "Point") and (geometry.geom_type.iloc[0] != "Polygon"):
        raise ValueError("Argument geometry must be either a Point or a Polygon")

    if (geometry.geom_type.iloc[0] != "Point") and (spatial_api == "point"):
        raise ValueError("Argument geometry must be a Point since argument spatial_api is point. Try to use pynasapower.geometry.point() to create a valid point.")
    
    if (geometry.geom_type.iloc[0] != "Polygon") and (spatial_api == "regional"):
        raise ValueError("Argument geometry must be a Polygon since argument spatial_api is regional.Try to use pynasapower.geometry.bbox() to create a valid polygon.")
    
    if geometry.geom_type.iloc[0] == "Point":
        latitude = geometry.geometry.y.iloc[0]
        longitude = geometry.geometry.x.iloc[0]
        coordinates = {
            "latitude": latitude,
            "longitude": longitude,
            }

    if geometry.geom_type.iloc[0] == "Polygon":
        latitude_min = geometry.bounds.miny.iloc[0]
        latitude_max = geometry.bounds.maxy.iloc[0]
        longitude_min = geometry.bounds.minx.iloc[0]
        longitude_max = geometry.bounds.maxx.iloc[0]
        
        if latitude_max - latitude_min < 2:
            raise ValueError("Please provide at least a 2 degree range in latitude; otherwise use the point endpoint.")
        
        if longitude_max - longitude_min < 2:
            raise ValueError("Please provide at least a 2 degree range in longitude; otherwise use the point endpoint.")

        coordinates = {
            "latitude-min": latitude_min,
            "latitude-max": latitude_max,
            "longitude-min": longitude_min,
            "longitude-max": longitude_max,
            }
    

    if not isinstance(start, datetime.date):
        raise TypeError("Argument 'start' must be datetime.date object.")    
    
    if not isinstance(end, datetime.date):
        raise TypeError("Argument 'end' must be datetime.date object.")
    
    if start > end:
        raise ValueError("Argument 'start' cannot be later than 'end'!")
    
    if temporal_api in ["monthly", "climatology"]:
        start_date = start.strftime("%Y")
        end_date = end.strftime("%Y")
        if temporal_api == "climatology":
            if int(end_date) - int(start_date) < 2:
                raise ValueError("Please provide at least a two year range to compute a climatology.") 
    else:
        start_date = start.strftime("%Y%m%d")
        end_date = end.strftime("%Y%m%d")
    
    server = f"https://power.larc.nasa.gov/api/temporal/{temporal_api}/{spatial_api}?"

    params = {
            "parameters": ",".join(parameters),
            "community": community,
            "start": start_date,
            "end": end_date,
            "format": format
            }
    
    params.update(coordinates)
    
    print ("Starting retrieval from NASA Power...")
    request = requests.get(server, params = params)
    # Check if server didn't respond to HTTP code = 200
    if request.status_code != HTTP_OK:
        raise exceptions.HTTPError(f"Failed retrieving POWER data, server returned HTTP code: {request.status_code} on following URL {request.url}.")
    # In other case is successful
    print ("Successfully retrieved data from NASA Power!")
    
    name = request.headers['content-disposition'].split("=")[1]
    
    if format == "netcdf":
        data = xr.open_dataset(request.content)
        data.to_netcdf(os.path.join(path, name))
    elif format == "csv" or format == "ascii":
        string = request.content.decode("utf-8")
        # Split the data into header and actual data
        header, string = string.split("-END HEADER-")
        data_lines = string.strip().split('\n')

        # Create a StringIO object for the data and load it into a DataFrame
        data_io = StringIO("\n".join(data_lines))
        data = pd.read_csv(data_io)
        
        if to_file:
            # Write header to txt
            txt_name = name.split(".")[0] + "_variables.txt"
            with open(os.path.join(path, txt_name), "w") as dst:
                dst.write(header)
            if format == "ascii":
                data.to_csv(os.path.join(path, name), sep='\t', index=False)
            else:
                data.to_csv(os.path.join(path, name), index=False)
    else:
        string = request.content.decode("utf-8")
        with open(os.path.join(path, name), "w") as outfile:
            outfile.write(string)
        
        data = request.json()
    
    return data

In [ ]:
https://github.com/alekfal/pynasapower

In [13]:
output_path = r"D:/X"


In [9]:
# Definir el bounding box para un área pequeña en Peru
gbbox = bbox(-74.89, -68.39, -9.09, 0.21, "EPSG:4326")  # bbox(-74.89, -68.39, -18.39, -9.09, "EPSG:4326")
"""
  lon_min lat_min lon_max lat_max                       geometry
1  -81.39  -18.39  -74.89   -9.09 POLYGON ((-81.39 -18.39, -7...  second cuadro
2  -81.39   -9.09  -74.89    0.21 POLYGON ((-81.39 -9.09, -74...  third picture
3  -74.89  -18.39  -68.39   -9.09 POLYGON ((-74.89 -18.39, -6...   frist cuadro
4  -74.89   -9.09  -68.39    0.21 POLYGON ((-74.89 -9.09, -68...   forty picture
"""

# Definir el rango de años
start_year = 1981  # Año de inicio
end_year = 2023    # Año final

# Especificar la ruta de la carpeta donde se guardarán los datos
output_path = r"D:/X/power4"  # Cambia esto a la ruta de tu carpeta deseada

# Lista de parámetros a descargar (ejemplo: T2M, RH2M)
parameters = ["PRECTOTCORR", "T2M_MAX", "T2M_MIN"]  # Añade aquí los parámetros que necesites
               # ["ALLSKY_SFC_SW_DWN", "WS2M", "T2M", "T2M_MIN", "T2M_MAX", "RH2M", "WS2M", "PRECTOTCORR"]
# Descargar datos para cada año
for year in range(start_year, end_year + 1):
    start = datetime.date(year, 1, 1)
    end = datetime.date(year, 12, 31)

    for param in parameters:
        try:
            # Realizar la consulta y guardar los datos en formato NetCDF
            data = query_power(
                geometry=gbbox,
                start=start,
                end=end,
                path=output_path,
                to_file=True,
                community="ag",
                parameters=parameters,
                #parameters=[param],  # Parámetro individual
                temporal_api="daily",
                spatial_api="regional",
                format="netcdf"
            )
            print(f'Datos diarios de {param}. del {start} al {end} guardados en {output_path}.')
        except Exception as e:
            print(f'Error al descargar datos de {param} para el año {year}: {e}')


Starting retrieval from NASA Power...
Error al descargar datos de PRECTOTCORR para el año 1981: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=19810101&end=19811231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MAX para el año 1981: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=19810101&end=19811231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MIN para el año 1981: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.

Error al descargar datos de PRECTOTCORR para el año 1988: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=19880101&end=19881231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MAX para el año 1988: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=19880101&end=19881231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MIN para el año 1988: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regio

Error al descargar datos de PRECTOTCORR para el año 1995: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=19950101&end=19951231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MAX para el año 1995: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=19950101&end=19951231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MIN para el año 1995: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regio

Error al descargar datos de PRECTOTCORR para el año 2002: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20020101&end=20021231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MAX para el año 2002: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20020101&end=20021231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MIN para el año 2002: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regio

Error al descargar datos de PRECTOTCORR para el año 2009: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20090101&end=20091231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MAX para el año 2009: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20090101&end=20091231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MIN para el año 2009: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regio

Error al descargar datos de PRECTOTCORR para el año 2016: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20160101&end=20161231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MAX para el año 2016: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20160101&end=20161231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MIN para el año 2016: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regio

Error al descargar datos de PRECTOTCORR para el año 2023: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20230101&end=20231231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MAX para el año 2023: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR%2CT2M_MAX%2CT2M_MIN&community=ag&start=20230101&end=20231231&format=netcdf&latitude-min=-18.39&latitude-max=-0.04&longitude-min=-81.39&longitude-max=-68.65.
Starting retrieval from NASA Power...
Error al descargar datos de T2M_MIN para el año 2023: Failed retrieving POWER data, server returned HTTP code: 422 on following URL https://power.larc.nasa.gov/api/temporal/daily/regio

In [12]:
import xarray as xr
import os

# Define el directorio principal y las subcarpetas para cada región
directorio_principal = "D:/X/"
regiones = ["power", "power2", "power3", "power4"]  # Nombres de carpetas para cada región

# Rango de años
start_year = 1981
end_year = 2023

# Directorio de salida para los archivos NetCDF unidos
output_directorio = "D:/X/PowerNasaPeru/"

# Asegúrate de que el directorio de salida existe
os.makedirs(output_directorio, exist_ok=True)

# Procesar cada año
for year in range(start_year, end_year + 1):
    archivos_anuales = []
    
    # Recolecta los archivos NetCDF de cada subregión para el año actual
    for region in regiones:
        region_path = os.path.join(directorio_principal, region)
        
        # Busca archivos para el año actual en la carpeta de la región
        archivos_region = [os.path.join(region_path, f) for f in os.listdir(region_path) if f"{year}" in f and f.endswith('.nc')]
        archivos_anuales.extend(archivos_region)
    
    # Verifica si hay archivos para combinar
    if archivos_anuales:
        # Abrir y combinar los datasets de subregiones
        datasets = [xr.open_dataset(f) for f in archivos_anuales]
        
        # Combina las subregiones en un solo dataset
        ds_unido = xr.combine_by_coords(datasets, combine_attrs='drop_conflicts')
        
        # Guardar el archivo NetCDF unido
        output_filename = f"Nasa Power Peru_{year}.nc"
        ds_unido.to_netcdf(os.path.join(output_directorio, output_filename))
        
        print(f"Subregiones unidas para el año {year} y guardadas en {output_filename}")
        
    else:
        print(f"No se encontraron archivos para el año {year} en las regiones especificadas.")


Subregiones unidas para el año 1981 y guardadas en Nasa Power Peru_1981.nc
Subregiones unidas para el año 1982 y guardadas en Nasa Power Peru_1982.nc
Subregiones unidas para el año 1983 y guardadas en Nasa Power Peru_1983.nc
Subregiones unidas para el año 1984 y guardadas en Nasa Power Peru_1984.nc
Subregiones unidas para el año 1985 y guardadas en Nasa Power Peru_1985.nc
Subregiones unidas para el año 1986 y guardadas en Nasa Power Peru_1986.nc
Subregiones unidas para el año 1987 y guardadas en Nasa Power Peru_1987.nc
Subregiones unidas para el año 1988 y guardadas en Nasa Power Peru_1988.nc
Subregiones unidas para el año 1989 y guardadas en Nasa Power Peru_1989.nc
Subregiones unidas para el año 1990 y guardadas en Nasa Power Peru_1990.nc
Subregiones unidas para el año 1991 y guardadas en Nasa Power Peru_1991.nc
Subregiones unidas para el año 1992 y guardadas en Nasa Power Peru_1992.nc
Subregiones unidas para el año 1993 y guardadas en Nasa Power Peru_1993.nc
Subregiones unidas para e